# Filling in Missing Values in Tabular Records

You can select Run->Run All Cells from the menu to run all cells in Studio (or Cell->Run All in a SageMaker Notebook Instance).

## Introduction

Missing data values are common due to omissions during manual entry or optional input. Simple data imputation such as using the median/mode/average may not be satisfactory. When there are many features, we can sometimes train a model to use the existing features to predict the desired feature. 

This solution provides and end-to-end example that takes a tabular data set with a target column, trains and deploys an endpoint, and calls that endpoint to make predictions.

## Architecture
As part of the solution, the following services are used:

* Amazon S3: Used to store datasets.
* Amazon SageMaker Notebook: Used to preprocess and process the data, and to train the deep learning model.
* Amazon SageMaker Endpoint: Used to deploy the trained model.

![](docs/architecture.png)

## Data Set
We will use public data from the City of Cincinnati Public Services describing Fleet Inventory. We will train a model to predict missing values of a 'target' column based on the other columns.

Please see.
https://www.cincinnati-oh.gov/public-services/about-public-services/fleet-services/
https://data.cincinnati-oh.gov/Thriving-Neighborhoods/Fleet-Inventory/m8ba-xmjz

## Acknowledgements
AutoPilot code based on
https://github.com/aws/amazon-sagemaker-examples/blob/master/autopilot/sagemaker_autopilot_direct_marketing.ipynb

In [ ]:
# Replace these with your train/test CSV data and target columns. 
# If left empty, the sample data set will be used.
data_location = ''    # Ex. s3://your_bucket/your_file.csv
target = ''           # Specify target column name

if data_location == '':
    # Use sample dataset.
    dataset_file = 'data/dataset.csv'
    target = 'ASSET_TYPE'
else:
    # Download custom dataset.
    !aws s3 cp $data_location data/custom_dataset.csv
    print('Downloaded custom dataset')
    dataset_file = 'data/custom_dataset.csv'

## Inspect the Data

In [ ]:
import pandas as pd

data = pd.read_csv(dataset_file)
data

## Preprocess Data
Some of the entries in the target column are null. We will remove those entries for training/testing.

In [ ]:
import numpy as np

def remove_null_rows(data, target):
    idx = data[target].notna()
    return data.loc[idx]

def split_train_test(data, p=.9):
    idx = np.random.choice([True, False], replace = True, size = len(data), p=[.8, .2])
    train_df = data.iloc[idx]
    test_df = data.iloc[[not i for i in idx]]
    return train_df, test_df

In [ ]:
non_null_data = remove_null_rows(data, target)
train, test = split_train_test(non_null_data)

train_file = 'data/train.csv'
test_file = 'data/test.csv'

train.to_csv(train_file, index=False, header=True)
test.to_csv(test_file, index=False, header=True)

## Store Processed Data on S3

Now that we have our data in files, we store this data to S3 so we can use SageMaker AutoPilot.

In [ ]:
import sagemaker
from sagemaker.s3 import S3Uploader
import json

with open('stack_outputs.json') as f:
    sagemaker_configs = json.load(f)
    
s3_bucket = sagemaker_configs['S3Bucket']

train_data_s3_path = S3Uploader.upload(train_file, 's3://{}/data'.format(s3_bucket))
print('Train data uploaded to: ' + train_data_s3_path)
test_data_s3_path = S3Uploader.upload(test_file, 's3://{}/data'.format(s3_bucket))
print('Test data uploaded to: ' + test_data_s3_path)

### Configure AutoPilot

For the purposes of a demo, we will use only 2 candidates. Remove this parameter to run AutoPilot with its defaults (note: for this data set a full run will take ~ 4 several hours.)

In [ ]:
input_data_config = [{
      'DataSource': {
        'S3DataSource': {
          'S3DataType': 'S3Prefix',
          'S3Uri': 's3://{}/data/train'.format(s3_bucket)
        }
      },
      'TargetAttributeName': target
}]

output_data_config = {
    'S3OutputPath': 's3://{}/data/output'.format(s3_bucket)
  }
automl_job_config ={
          'CompletionCriteria': {
              'MaxCandidates': 2  # Remove this option for the default run.
          }
}


In [ ]:
import boto3 
from sagemaker import get_execution_role
from time import gmtime, strftime, sleep

role = get_execution_role()

solution_prefix = sagemaker_configs['SolutionPrefix']

auto_ml_job_name = solution_prefix + strftime('%d-%H-%M-%S', gmtime())
print('AutoMLJobName: ' + auto_ml_job_name)

sm = boto3.Session().client(service_name='sagemaker',region_name='us-west-2')
sm.create_auto_ml_job(AutoMLJobName=auto_ml_job_name,
                      InputDataConfig=input_data_config,
                      OutputDataConfig=output_data_config,
                      AutoMLJobConfig=automl_job_config,
                      RoleArn=role)

In [ ]:
# This will take approximately 20 minutes to run.
secondary_status = ''
while True:
    describe_response = sm.describe_auto_ml_job(AutoMLJobName=auto_ml_job_name)
    job_run_status = describe_response['AutoMLJobStatus']
    
    if job_run_status in ('Failed', 'Completed', 'Stopped'):
        print('\n{}: {}'.format(describe_response['AutoMLJobSecondaryStatus'], job_run_status))
        break

    if secondary_status == describe_response['AutoMLJobSecondaryStatus']:
        print('.', end='')        
    else:
        secondary_status = describe_response['AutoMLJobSecondaryStatus']
        print('\n{}: {}'.format(secondary_status, job_run_status), end='')
    
    sleep(60)

In [ ]:
best_candidate = sm.describe_auto_ml_job(AutoMLJobName=auto_ml_job_name)['BestCandidate']
best_candidate_name = best_candidate['CandidateName']
print(best_candidate)
print('\n')
print("CandidateName: " + best_candidate_name)
print("FinalAutoMLJobObjectiveMetricName: " + best_candidate['FinalAutoMLJobObjectiveMetric']['MetricName'])
print("FinalAutoMLJobObjectiveMetricValue: " + str(best_candidate['FinalAutoMLJobObjectiveMetric']['Value']))

In [ ]:
model_name = sagemaker_configs['SageMakerModelName']

model = sm.create_model(Containers=best_candidate['InferenceContainers'],
                            ModelName=model_name,
                            ExecutionRoleArn=role)


## Deploy and Endpoint

In [ ]:
print("Building endpoint with model {}".format(model))

In [ ]:
endpoint_config_name = sagemaker_configs['SageMakerEndpointName'] + '-config'
create_endpoint_config_response = sm.create_endpoint_config(
    EndpointConfigName = endpoint_config_name,
    ProductionVariants=[{
        'InstanceType':'ml.m5.xlarge',
        'InitialVariantWeight':1,
        'InitialInstanceCount':1,
        'ModelName':model_name,
        'VariantName':'AllTraffic'}])

In [ ]:
endpoint_name = sagemaker_configs['SageMakerEndpointName']
create_endpoint_response = sm.create_endpoint(
    EndpointName=endpoint_name,
    EndpointConfigName=endpoint_config_name,
    )
print(create_endpoint_response['EndpointArn'])

In [ ]:
resp = sm.describe_endpoint(EndpointName=endpoint_name)
status = resp['EndpointStatus']
print("Status: " + status)

In [ ]:
import time

print('Creating Endpoint... this may take several minutes')
while status=='Creating':
    resp = sm.describe_endpoint(EndpointName=endpoint_name)
    status = resp['EndpointStatus']
    print('.', end='')
    time.sleep(15) 
print("\nStatus: " + status)

## Test the Endpoint

In [ ]:
runtime_client = boto3.client('runtime.sagemaker')

test_input = test.drop(columns=[target])[0:10]
test_input_csv = test_input.to_csv(index=False, header=False).split('\n')
test_labels = test[target][0:10]



for i, (single_test, single_label) in enumerate(zip(test_input_csv, test_labels)):
    print('=== Test {} ===\nInput: {}\n'.format(i, single_test))    
    response = runtime_client.invoke_endpoint(EndpointName = endpoint_name,
                                        ContentType = 'text/csv',
                                        Body = single_test)
    result = response['Body'].read().decode('ascii')
    print('Predicted label is {}\nCorrect label is {}\n'.format(result.rstrip(), single_label.rstrip()))    

## Clean up

Stack deletion will clean up all created resources including S3 buckets, Endpoint configurations, Endpoints and Models.